In [127]:
import nltk
from nltk.chat.util import Chat, reflections
import json
import re
import unicodedata
import random
import json
import os
import tkinter as tk
from tkinter import scrolledtext, messagebox
from datetime import datetime
from nltk.chat.util import Chat, reflections

In [128]:

#Archivo donde se guarda el historial
HISTORIAL_FILE = "historial.json"

In [129]:
def normalizar_texto(texto):
    """
    Convierte texto a minúsculas y elimina tildes para mejorar la coincidencia de patrones.
    """
    texto = texto.lower()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')  # Elimina tildes
    return texto

In [130]:
def leer_json_y_generar_patrones_respuestas(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        data = json.load(archivo)
    
    patrones_respuestas = []
    patrones_unificados = {}  # Para agrupar patrones similares en una sola expresión

    for intent in data['intents']:
        patterns = intent['patterns']
        responses = intent['responses']
        
        # Normalizamos los patrones para evitar problemas con tildes o mayúsculas
        patterns_normalizados = [normalizar_texto(p) for p in patterns]
        regex_pattern = r'\b(' + '|'.join(map(re.escape, patterns_normalizados)) + r')\b'
        
        # Agrupamos respuestas si ya existe un patrón similar
        if regex_pattern in patrones_unificados:
            patrones_unificados[regex_pattern].extend(responses)
        else:
            patrones_unificados[regex_pattern] = responses

    # Convertimos el diccionario en la lista de tuplas (pattern, responses)
    for pattern, responses in patrones_unificados.items():
        patrones_respuestas.append((pattern, responses))

    return patrones_respuestas

In [131]:
# Ruta del archivo JSON
ruta_archivo = '..\\datos web\\output.json'
patrones_respuestas = leer_json_y_generar_patrones_respuestas(ruta_archivo)

for item in patrones_respuestas:
    print(item)

('\\b(contacto)\\b', ['Correo: 46019015@edu.gva.es\nFAX : 961206091\nE-mail: 46019015.secretaria@edu.gva.es\nTelefono : 961206090'])
('\\b(correo)\\b', ['Correo: 46019015@edu.gva.es\nE-mail: 46019015.secretaria@edu.gva.es'])
('\\b(fax)\\b', ['FAX : 961206091'])
('\\b(telefono)\\b', ['Telefono : 961206090'])
('\\b(¿que\\ informacion\\ hay\\ sobre\\ paginas\\ web\\?)\\b', ['http://www.dmae.upct.es/~juan/matematicas.htm', 'https://areatablet.wordpress.com/', 'http://www.ceice.gva.es/doc046/gen/es/tasas.asp?idf=TITULOS', 'https://ceice.gva.es/es/web/formacion-profesional/pruebas-de-acceso-a-ciclos-formativos', 'https://media.upv.es/#/catalog/channel/eda03965-19d6-1a4e-abaf-cf5b04ce1a6e', 'http://www.intersindical.org/escola/activitat/9081', 'https://portal.edu.gva.es/46019015/', 'http://cefire.edu.gva.es/sfp/index.php?usuario=formacion', 'http://www.ceice.gva.es/doc046/gen/val/tasas.asp?idf = ADMIN', 'http://www.apuntesmareaverde.org.es/', 'http://www.ceice.gva.es/doc046/gen/val/tasas.asp?

In [132]:
# Crear el chatbot usando `nltk.Chat`
chatbot = Chat(patrones_respuestas, reflections)


In [133]:
def estilo_boton(boton):
    boton.config(bg="#3498DB", fg="white", font=("Arial", 10, "bold"), relief="flat", bd=0, padx=10, pady=5)
    boton.bind("<Enter>", lambda e: boton.config(bg="#2980B9"))
    boton.bind("<Leave>", lambda e: boton.config(bg="#3498DB"))

In [134]:
#Cargar historial desde JSON
def cargar_historial():
    if os.path.exists(HISTORIAL_FILE):
        with open(HISTORIAL_FILE, "r", encoding="utf-8") as file:
            return json.load(file)
    return {}

In [135]:

#Guardar historial en JSON
def guardar_historial(historial):
    with open(HISTORIAL_FILE, "w", encoding="utf-8") as file:
        json.dump(historial, file, indent=4, ensure_ascii=False)

In [136]:
#Verifica si el usuario existe y muestra su historial
def iniciar_sesion():
    nombre = entry_nombre.get().strip()
    if not nombre:
        messagebox.showwarning("Error", "Por favor, ingrese su nombre")
        return

    historial = cargar_historial()
    if nombre not in historial:
        historial[nombre] = []  #Crear historial vacío para nuevos usuarios
        guardar_historial(historial)

    ventana_login.destroy()  #Cerrar la ventana de login
    mostrar_chat(nombre, historial)

In [137]:
#Ventana de historial
def mostrar_historial(nombre, historial):
    ventana_historial = tk.Toplevel()
    ventana_historial.title(f"Historial de {nombre}")
    ventana_historial.geometry("400x400")
    ventana_historial.configure(bg="#2C3E50")

    historial_text = scrolledtext.ScrolledText(ventana_historial, wrap=tk.WORD, width=50, height=20, bg="#ECF0F1", fg="#2C3E50", font=("Arial", 10))
    historial_text.pack(padx=10, pady=10)
    historial_text.insert(tk.END, "\n".join(historial[nombre]) + "\n")  #Cargar historial completo
    historial_text.config(state=tk.DISABLED)

In [138]:
#Eliminar historial de usuario
def borrar_historial(nombre, historial, chat_text):
    if messagebox.askyesno("Borrar Historial", "¿Estás seguro de que quieres borrar tu historial?"):
        historial[nombre] = []
        guardar_historial(historial)
        chat_text.config(state=tk.NORMAL)
        chat_text.delete(1.0, tk.END)
        chat_text.config(state=tk.DISABLED)
        messagebox.showinfo("Historial Borrado", "Tu historial ha sido eliminado correctamente.")


In [139]:
#Interfaz de chat con historial
def mostrar_chat(nombre, historial):
    ventana_chat = tk.Tk()
    ventana_chat.title(f"Chat de {nombre}")
    ventana_chat.geometry("400x500")
    ventana_chat.configure(bg="#2C3E50")

    chat_text = scrolledtext.ScrolledText(ventana_chat, wrap=tk.WORD, width=50, height=15, bg="#ECF0F1", fg="#2C3E50", font=("Arial", 10))
    chat_text.pack(padx=10, pady=10)
    chat_text.insert(tk.END, "\n".join(historial[nombre]) + "\n")  #Cargar historial reciente
    chat_text.config(state=tk.DISABLED)

    entrada_texto = tk.Entry(ventana_chat, width=40, font=("Arial", 10), bg="#ECF0F1", fg="#2C3E50")
    entrada_texto.pack(pady=5)

    def enviar_mensaje():
        mensaje = entrada_texto.get().strip()
        user_input_normalizado = normalizar_texto(mensaje)
        if user_input_normalizado:
            if user_input_normalizado in ["exit", "quit"]:
                ventana_chat.destroy()  # Close the chat window
                return

            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            chat_text.config(state=tk.NORMAL)
            chat_text.insert(tk.END, f"\n{timestamp} You: {user_input_normalizado}\n")
            chat_text.config(state=tk.DISABLED)

            response = None
            for pattern, respuestas in patrones_respuestas:
                if re.search(pattern, user_input_normalizado, re.IGNORECASE):  # Busca sin importar mayúsculas o minúsculas
                    response = random.choice(respuestas)
                    break

            if response:
                chat_text.config(state=tk.NORMAL)
                chat_text.insert(tk.END, f"{timestamp} SerraBot: {response}\n")
                chat_text.config(state=tk.DISABLED)
                historial[nombre].append(f"{timestamp} You: {user_input_normalizado}")
                historial[nombre].append(f"{timestamp} SerraBot: {response}")
                guardar_historial(historial)

            entrada_texto.delete(0, tk.END)


    boton_enviar = tk.Button(ventana_chat, text="Enviar", command=enviar_mensaje, bg="#3498DB", fg="white", font=("Arial", 10, "bold"), height=2, width=6)
    estilo_boton(boton_enviar)
    boton_enviar.pack(pady=15)

    boton_historial = tk.Button(ventana_chat, text="Ver Historial Completo", command=lambda: mostrar_historial(nombre, historial), bg="#E74C3C", fg="white", font=("Arial", 10, "bold"))
    estilo_boton(boton_historial)
    boton_historial.pack(pady=5)

    boton_borrar = tk.Button(ventana_chat, text="Borrar Historial", command=lambda: borrar_historial(nombre, historial, chat_text), bg="#C0392B", fg="white", font=("Arial", 10, "bold"))
    estilo_boton(boton_borrar)
    boton_borrar.pack(pady=5)

    ventana_chat.mainloop()

In [140]:

#Ventana de inicio de sesión
ventana_login = tk.Tk()
ventana_login.title("Inicio de Sesión")
ventana_login.geometry("450x350")
ventana_login.configure(bg="#2C3E50")

tk.Label(ventana_login, text="Bienvenido a SerraBot", fg="white", bg="#2C3E50", font=("Arial", 12, "bold")).pack(pady=10)
tk.Label(ventana_login, text="Ingrese su nombre:", fg="white", bg="#2C3E50", font=("Arial", 12, "bold")).pack(pady=10)
entry_nombre = tk.Entry(ventana_login, font=("Arial", 10), bg="#ECF0F1", fg="#2C3E50")
entry_nombre.pack()


btn_ingresar = tk.Button(ventana_login, text="Acceder al chat", command=iniciar_sesion, bg="#27AE60", fg="white", font=("Arial", 10, "bold"))
estilo_boton(btn_ingresar)
btn_ingresar.pack(pady=10)

ventana_login.mainloop()